## Installing Necessary Libraries

In [10]:
!pip install -i https://test.pypi.org/simple/ bitsandbytes

Looking in indexes: https://test.pypi.org/simple/


In [11]:
!pip install accelerate

In [1]:
!pip install wandb trl transformers accelerate peft datasets bitsandbytes einops pdfquery timeout-decorator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [1]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from google.colab import userdata
wandb_secret = userdata.get('wandb')

In [4]:
import os
# Set the environment variable
os.environ['wandb_secret'] = wandb_secret

In [5]:
!wandb login $wandb_secret

Traceback (most recent call last):
  File "/usr/local/bin/wandb", line 8, in <module>
    sys.exit(cli())
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1078, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1688, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1434, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 783, in invoke
    return __callback(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/wandb/cli/cli.py", line 103, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/wandb/cli/cli.py", line 247, in login
    wandb.login(relogin=relogin, key=key, anonymo

In [6]:
import wandb
wandb.init()

wandb: Currently logged in as: digvijay-2-ok. Use `wandb login --relogin` to force relogin


In [7]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

# model_name = "NousResearch/Llama-2-7b-hf"
model_name = "NousResearch/Llama-2-7b-chat-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [9]:
import transformers
from datasets import Dataset

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    # torch_dtype=torch.bfloat16,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
)

In [10]:
import seaborn as sns
import pandas as pd
import numpy as np
import gc
import time
import timeout_decorator

In [11]:
@timeout_decorator.timeout(60)
def predict_response(text):
    predictions = pipeline(text, max_length=300,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id)

    # Print the predictions
    # print(predictions)
    mresponse = predictions[0]['generated_text']
    return mresponse

In [12]:
from datasets import load_dataset
dataset = load_dataset("abhishek-mungoli/srk_wittiness")
df = pd.DataFrame({'question': dataset['train']['question'],'response': dataset['train']['response']})
display(df)

,question,response
0,If you could erase one project from your caree...,"Well, I would erase the project of trying to g..."
1,Can you share a moment when you doubted your o...,"Well, there was this one time when I doubted m..."
2,What's the most significant criticism you've r...,"Well, someone once told me I overact. I took i..."
3,If you could trade talents with another celebr...,"Well, I would trade talents with Leonardo DiCa..."
4,How do you handle negative feedback or critici...,"Well, I treat it like I treat my morning alarm..."
...,...,...
206,What books do you read?,"Well, I mostly read scripts. They're like nove..."
207,What things do you want to change in the indus...,"Well, I would like to change the perception th..."
208,Could you live a good life without privacy?,"Well, I've been living in the public eye for s..."
209,Tell us about your new family movie.,"Well, it's a movie where I play a father who i..."


In [13]:
from sklearn.model_selection import train_test_split

X = df.drop("response", axis=1)
y = df["response"]

# Split the data into a training set (80%) and a test set (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.18, random_state=42)

# Create new DataFrames df_train and df_test
df_train = pd.concat([X_train, y_train], axis=1)
df_test = pd.concat([X_test, y_test], axis=1)


print( df_train.shape, df_test.shape )

(173, 2) (38, 2)


In [14]:
df_train['text'] = df_train.apply(lambda row: f"Question: {row['question']}\nShah Rukh Khan: {row['response']}", axis=1)
df_test['text'] = df_test.apply(lambda row: f"Question: {row['question']}\nShah Rukh Khan: ", axis=1)

In [15]:
display(df_train)

,question,response,text
189,Have you ever dated a fan?,"Well, I've been dating my biggest fan for over...",Question: Have you ever dated a fan?\nShah Ruk...
93,If you had to spend a day as an inanimate obje...,"I would choose to be a mirror. That way, I cou...",Question: If you had to spend a day as an inan...
115,"You're the captain of a spaceship, and you hav...","Well, I would definitely choose Iron Man for h...","Question: You're the captain of a spaceship, a..."
163,Were you a part of any school clubs or organiz...,"Well, I was a part of the 'I'm too cool for sc...",Question: Were you a part of any school clubs ...
195,What’s a message you want to tell your fans?,"Well, I would say, keep loving me as you do, a...",Question: What’s a message you want to tell yo...
...,...,...,...
106,If you could communicate with extraterrestrial...,"Well, I'd probably ask them if they've seen an...",Question: If you could communicate with extrat...
14,How do you handle situations where you strongl...,"Well, I usually handle it by reminding myself ...",Question: How do you handle situations where y...
92,"In a zombie apocalypse, which three celebritie...","Well, I would choose Bear Grylls for survival ...","Question: In a zombie apocalypse, which three ..."
179,What's a lesson you learned from a friendship ...,"Well, I learned that if you want to keep your ...",Question: What's a lesson you learned from a f...


In [16]:
df_test.shape

(38, 3)

In [17]:
infer_limit = 30
mix = 0
solutions = []
for ix, row in df_test.iterrows():
    try:
        text = f"Shah Rukh Khan is an actor known for his witty responses. How would he reply with wit and humor to the following question - {row['question']}"
        solutions.append([row['question'], predict_response(text), row['response']])
        print(mix, "Done")
    except Exception as e:
        print("An exception occurred:", str(e))
        try:
            solutions.append([row['question'], predict_response(text), row['response']])
            print(mix, "Done")
        except Exception as e:
            print("An exception occurred:", str(e))
    # if mix > infer_limit:
    #     break
    mix += 1

df_soln = pd.DataFrame(solutions, columns=['question','llama_response', 'chatgpt_response'])

display(df_soln)

df_soln.to_csv('/content/drive/MyDrive/Data Science/output_unmerge.csv', index=False, header=True)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


0 Done
An exception occurred: 'Timed Out'
1 Done
2 Done
3 Done
4 Done
5 Done
An exception occurred: 'Timed Out'
6 Done
7 Done


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


An exception occurred: 'Timed Out'
An exception occurred: 'Timed Out'
An exception occurred: 'Timed Out'
9 Done
10 Done
An exception occurred: 'Timed Out'
An exception occurred: 'Timed Out'
An exception occurred: 'Timed Out'
12 Done
13 Done
An exception occurred: 'Timed Out'
14 Done
15 Done
16 Done
17 Done
An exception occurred: 'Timed Out'
An exception occurred: 'Timed Out'
An exception occurred: 'Timed Out'
An exception occurred: 'Timed Out'
An exception occurred: 'Timed Out'
20 Done
21 Done
22 Done
23 Done
An exception occurred: 'Timed Out'
24 Done
25 Done
An exception occurred: 'Timed Out'
26 Done
An exception occurred: 'Timed Out'
27 Done
An exception occurred: 'Timed Out'
28 Done
An exception occurred: 'Timed Out'
29 Done
An exception occurred: 'Timed Out'
An exception occurred: 'Timed Out'
An exception occurred: 'Timed Out'
An exception occurred: 'Timed Out'
32 Done
33 Done
An exception occurred: 'Timed Out'
34 Done
35 Done
An exception occurred: 'Timed Out'
36 Done
37 Done


,question,llama_response,chatgpt_response
0,What's the most unusual or unexpected source o...,Shah Rukh Khan is an actor known for his witty...,"Well, once I was inspired by my wife's shoppin..."
1,Did you ever participate in a talent show at s...,Shah Rukh Khan is an actor known for his witty...,"Oh yes, I did participate in a talent show at ..."
2,"What was your first job, and what did you lear...",Shah Rukh Khan is an actor known for his witty...,My first job was at a concert in Delhi where I...
3,Can you respond to the criticism that your rec...,Shah Rukh Khan is an actor known for his witty...,"Well, I've always believed in the philosophy o..."
4,Do you think you deserve all the accolades you...,Shah Rukh Khan is an actor known for his witty...,"Well, I believe I've worked hard for my accola..."
5,Could you live a good life without privacy?,Shah Rukh Khan is an actor known for his witty...,"Well, I've been living in the public eye for s..."
6,Can you recall a moment when you felt creative...,Shah Rukh Khan is an actor known for his witty...,"Well, there was a time when I felt my creativi..."
7,If you could relive one day from your childhoo...,Shah Rukh Khan is an actor known for his witty...,"Well, I would choose the day I was born. I mea..."
8,You find a magic lamp with a genie inside. Wha...,Shah Rukh Khan is an actor known for his witty...,"Well, firstly, I would wish for an unlimited s..."
9,"If you could time-travel to your childhood, wh...",Shah Rukh Khan is an actor known for his witty...,"I would tell my younger self, 'Don't worry, yo..."


OSError: Cannot save file into a non-existent directory: '/content/drive/MyDrive/Colab Notebooks'

In [18]:
from datasets import Dataset,DatasetDict
train_dataset_dict = DatasetDict({
    "train": Dataset.from_pandas(df_train),
})

In [19]:
from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    # target_modules=["q_proj","v_proj"]
    target_modules=['q_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj', 'k_proj', 'v_proj'] # Choose all linear layers from the model
)

In [20]:
!rm -r ./results

rm: cannot remove './results': No such file or directory


In [21]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 2
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 5
logging_steps = 1
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 300
# max_steps = 3
warmup_ratio = 0.03
lr_scheduler_type = "constant"
save_total_limit = 1

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    ddp_find_unused_parameters=False,
    save_total_limit = save_total_limit
)

In [22]:
from trl import SFTTrainer

max_seq_length = 300

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_dict['train'],
    # train_dataset=data['train'],
    peft_config=peft_config,
    dataset_text_field="text",
    # dataset_text_field="prediction",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

Map:   0%|          | 0/173 [00:00<?, ? examples/s]

In [23]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [24]:
!export TORCH_DISTRIBUTED_DEBUG=INFO

In [25]:
import torch
torch.cuda.empty_cache()

import gc
gc.collect()

227

In [26]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,2.016300
2,2.076500
3,2.085000
4,1.853200
5,2.058900
6,1.649200
7,1.541100
8,1.734800
9,1.653800
10,1.795200


TrainOutput(global_step=300, training_loss=0.31571250580251214, metrics={'train_runtime': 3446.3176, 'train_samples_per_second': 0.696, 'train_steps_per_second': 0.087, 'total_flos': 8147175234895872.0, 'train_loss': 0.31571250580251214, 'epoch': 13.79})

In [27]:
trainer.save_model("/content/drive/MyDrive/Data Science/llamav2-7b-lora-fine-tune_unmerge")

In [ ]:
# adapter_model_name = '/content/drive/MyDrive/Colab Notebooks/llamav2-7b-lora-fine-tune'
# base_model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     quantization_config=bnb_config,
#     trust_remote_code=True
# )

In [ ]:
# from peft import PeftConfig, PeftModel
# final_model = PeftModel.from_pretrained(base_model, adapter_model_name)

# final_model = final_model.merge_and_unload()

In [ ]:
# final_model.save_pretrained("/content/drive/MyDrive/Colab Notebooks/merged_adapters")

In [28]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    # torch_dtype=torch.bfloat16,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
)

In [29]:
@timeout_decorator.timeout(60)
def predict_response_finetuned(text):
    predictions = pipeline(text, max_length=300,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id)

    # Print the predictions
    # print(predictions)
    mresponse = predictions[0]['generated_text']
    return mresponse

In [30]:
import time

results = []

start_time = time.time()  # Record the start time

mix = 0
solutions = []
for ix, row in df_test.iterrows():
    try:
        # text = f"Shah Rukh Khan is an actor known for his witty responses. How would he reply with wit and humor to the following question - {row['question']}"
        text = row['text']
        solutions.append([row['question'], predict_response_finetuned(text), row['response']])
    except Exception as e:
        print("An exception occurred:", str(e))
        try:
            solutions.append([row['question'], predict_response_finetuned(text), row['response']])
        except Exception as e:
            print("An exception occurred:", str(e))
    # if mix > 2:
    #     break
    mix += 1

# This will print a warning message, but it will be more efficient than calling the pipeline sequentially.

end_time = time.time()  # Record the end time
elapsed_time = end_time - start_time

print("Time taken to run the loop:", elapsed_time/60.0, "minutes")

df_soln = pd.DataFrame(solutions, columns=['question','llama_response', 'chatgpt_response'])

display(df_soln)

df_soln.to_csv('/content/drive/MyDrive/Data Science/output_finetuned_unmerge.csv', index=False, header=True)

An exception occurred: 'Timed Out'


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Time taken to run the loop: 18.495084273815156 minutes


,question,llama_response,chatgpt_response
0,What's the most unusual or unexpected source o...,Question: What's the most unusual or unexpecte...,"Well, once I was inspired by my wife's shoppin..."
1,Did you ever participate in a talent show at s...,Question: Did you ever participate in a talent...,"Oh yes, I did participate in a talent show at ..."
2,"What was your first job, and what did you lear...","Question: What was your first job, and what di...",My first job was at a concert in Delhi where I...
3,Can you respond to the criticism that your rec...,Question: Can you respond to the criticism tha...,"Well, I've always believed in the philosophy o..."
4,Do you think you deserve all the accolades you...,Question: Do you think you deserve all the acc...,"Well, I believe I've worked hard for my accola..."
5,Could you live a good life without privacy?,Question: Could you live a good life without p...,"Well, I've been living in the public eye for s..."
6,Can you recall a moment when you felt creative...,Question: Can you recall a moment when you fel...,"Well, there was a time when I felt my creativi..."
7,If you could relive one day from your childhoo...,Question: If you could relive one day from you...,"Well, I would choose the day I was born. I mea..."
8,"If you had to start your career over, what wou...",Question: If you had to start your career over...,"Well, I would probably start by convincing my ..."
9,You find a magic lamp with a genie inside. Wha...,Question: You find a magic lamp with a genie i...,"Well, firstly, I would wish for an unlimited s..."


OSError: Cannot save file into a non-existent directory: '/content/drive/MyDrive/Colab Notebooks'

In [31]:
solutions = []

questions = ['Question: How is a neural network trained?\nShah Rukh Khan: ','Question: How to learn machine learning?\nShah Rukh Khan: ',
             'Question: I dont feel motivated to energetic to work, can you suggest me something?\nShah Rukh Khan: ']

for question in questions:
    try:
        text = question
        solutions.append(predict_response_finetuned(text))
    except Exception as e:
        print("An exception occurred:", str(e))
        try:
            solutions.append(predict_response_finetuned(text))
        except Exception as e:
            print("An exception occurred:", str(e))


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


An exception occurred: 'Timed Out'


In [32]:
solutions = []

questions = ['Question: How is a neural network trained?\nShah Rukh Khan: ','Question: How to learn machine learning?\nShah Rukh Khan: ',
             'Question: I dont feel motivated to energetic to work, can you suggest me something?\nShah Rukh Khan: ']

for question in questions:
    try:
        text = question
        solutions.append(predict_response_finetuned(text))
    except Exception as e:
        print("An exception occurred:", str(e))
        try:
            solutions.append(predict_response_finetuned(text))
        except Exception as e:
            print("An exception occurred:", str(e))

In [33]:
for ix,question in enumerate(questions):
  print(question, solutions[ix])
  print("----------------------------------")

Question: How is a neural network trained?
Shah Rukh Khan:  Question: How is a neural network trained?
Shah Rukh Khan:  Well, it's like training a muscle. You feed it a lot of data and then it learns to recognize patterns. And trust me, my neural network can spot a fake tan a mile away!
----------------------------------
Question: How to learn machine learning?
Shah Rukh Khan:  Question: How to learn machine learning?
Shah Rukh Khan:  Well, I would suggest you start by learning how to learn. After that, it's just a matter of time and practice. Oh, and don't forget to watch my movies. They are the best way to learn how to predict the future!
----------------------------------
Question: I dont feel motivated to energetic to work, can you suggest me something?
Shah Rukh Khan:  Question: I dont feel motivated to energetic to work, can you suggest me something?
Shah Rukh Khan:  Well, I would suggest you take a break and go watch one of my movies. After all, who better to motivate you than t